In [4]:
import sys
import torch
import numpy as np
import pickle as pkl
from tqdm import tqdm

sys.path.append('../../../')

from experiments.assumptions.degeneracy.script import eigenvalue_result, eigenvalue_results_large, plot_rank_train, rank_over_training
from models.supervised.mlp.model import MLP
from models.supervised.bimt.model import BioMLP
from models.unsupervised.vae.model import VAE, Encoder, Decoder


In [5]:
np.random.seed(2)
torch.manual_seed(2)

In [ ]:
mode = "moon"
model_name = "mlp"
size = "2_wide"
models_path = f"../../../models/supervised/{model_name}/saved_models"
if size == "overfit":
	with open(f'{models_path}/2_wide/{mode}/dataset.pkl', 'rb') as f:
		dataset = pkl.load(f)
else:
	with open(f'{models_path}/{size}/{mode}/dataset.pkl', 'rb') as f:
		dataset = pkl.load(f)

for size in ["mixup_vanilla", "vanilla",  "overfit", "2_wide"]:
	if size == "overfit":
		epochs = [60, 80, 199, 9999, 999, 5]
	else:
		epochs = [5, 60, 80, 199]
	for epoch in epochs:
		if size == "vanilla" or size == "mixup_vanilla":
			model = MLP(2,7,10,2)
		elif size == "overfit":
			model = MLP(2,7,2,1)
		elif size == "2_wide":
			model = MLP(2,7,2,2)
		print(f"Size: {size}, Epoch: {epoch}")
		model.eval()
		model.load_state_dict(torch.load(f'{models_path}/{size}/{mode}/model_{epoch}.pth'))
		if size == "overfit":
			model.num_layers -= 1
			model.layers = model.layers[:-1]
		save_path = f"figures/{model_name}/{mode}/{size}/{epoch}/"
		eigenvalue_result(dataset.X, model, N=50, labels=dataset.y, wrt="output_wise", sigma=0.05, precision=7, save_path=save_path)
		eigenvalue_result(dataset.X, model, N=50, labels=dataset.y, wrt="layer_wise", sigma=0.05, precision=7, save_path=save_path)

In [ ]:
mode = "blobs"
model_name = "mlp"
size = "vanilla"
with open(f'{models_path}/{size}/{mode}/dataset.pkl', 'rb') as f:
	dataset = pkl.load(f)
epochs = [5, 60, 80, 199]

for size in ["vanilla"]:
	for epoch in epochs:
		if size == "vanilla":
			model = MLP(2,7,4,4)
		print(f"Size: {size}, Epoch: {epoch}")
		model.eval()
		model.load_state_dict(torch.load(f'{models_path}/{size}/{mode}/model_{epoch}.pth'))
		save_path = f"figures/{model_name}/{mode}/{size}/{epoch}/"
		eigenvalue_result(dataset.X, model, N=50, labels=dataset.y, wrt="output_wise", sigma=0.05, precision=7, save_path=save_path)
		eigenvalue_result(dataset.X, model, N=50, labels=dataset.y, wrt="layer_wise", sigma=0.05, precision=7, save_path=save_path)

In [ ]:
mode="moon"
model_name = "bimt"
size = "vanilla"

model = BioMLP(shp=[2,20,20,2])
models_path = f"../../../models/supervised/{model_name}/saved_models"

res_q_25, res_med, res_q_75 = [], [], []
with open(f'{models_path}/{size}/{mode}/dataset.pkl', 'rb') as f:
	dataset = pkl.load(f)

for epoch in tqdm(range(0, 10000, 100)):
	model.load_state_dict(torch.load(f'{models_path}/{size}/{mode}/model_{epoch}.pth'))
	model.eval()
	save_path = f"figures/{model_name}/{mode}/{size}/{epoch}/"
	
	q_25, med, q_75 = eigenvalue_results_large(dataset.X, model, N=50, wrt="output_wise", sigma=0.05, precision=7, save_path=save_path)
	res_q_25.append(q_25)
	res_med.append(med)
	res_q_75.append(q_75)


	

In [ ]:
plot_rank_train(res_q_25, res_med, res_q_75, savepath=f"figures/{model_name}/{mode}/{size}/")


In [ ]:
mode="moon"
model_name = "mlp"
size = "overfit"


models_path = f"../../../models/supervised/{model_name}/saved_models"

res_q_25, res_med, res_q_75 = [], [], []
with open(f'{models_path}/2_wide/{mode}/dataset.pkl', 'rb') as f:
	dataset = pkl.load(f)

for epoch in tqdm(range(0, 10000, 100)):
	model = MLP(2,7,2,1)

	model.load_state_dict(torch.load(f'{models_path}/{size}/{mode}/model_{epoch}.pth'))
	if size == "overfit":
		model.num_layers -= 1
		model.layers = model.layers[:-1]
	model.eval()
	save_path = f"figures/{model_name}/{mode}/{size}/{epoch}/"
	
	q_25, med, q_75 = rank_over_training(dataset.X, model, N=50, wrt="output_wise", sigma=0.05, precision=7)
	res_q_25.append(q_25)
	res_med.append(med)
	res_q_75.append(q_75)

In [ ]:
plot_rank_train(res_q_25, res_med, res_q_75, savepath=f"figures/{model_name}/{mode}/{size}/")


In [11]:
!ls ../../../models/unsupervised/vae/saved_models/disentangled_3

Final                              disentangled
L0.1_C0_lr0.001_lam_sstep_lr_snone disentangled_2
L0.1_C0_lr0.002_lam_sstep_lr_sexp  disentangled_3


In [12]:
mode="Final"
model_name = "vae"
size = "disentangled_3"


models_path = f"../../../models/unsupervised/{model_name}/saved_models"
res_q_25, res_med, res_q_75 = [], [], []
with open(f'{models_path}/{size}/dataset.pkl', 'rb') as f:
	dataset = pkl.load(f)


In [13]:
features = [64, 32, 16, 8, 4]



In [14]:
for epoch in tqdm(range(0, 3200, 100)):
	encoder = Encoder(in_features=32, features=features, out_features=2)
	decoder = Decoder(in_features=2, features=list(reversed(features)), out_features=32)
	model = VAE(encoder, decoder)
	
	model.load_state_dict(torch.load(f'{models_path}/{size}/model_{epoch}.pth'))
	if size == "overfit":
		model.num_layers -= 1
		model.layers = model.layers[:-1]
	model.eval()
	save_path = f"figures/{model_name}/{mode}/{size}/{epoch}/"
	
	q_25, med, q_75 = rank_over_training(dataset.X.detach().numpy()[:1000], model, N=50, wrt="output_wise", sigma=0.05, precision=7)
	res_q_25.append(q_25)
	res_med.append(med)
	res_q_75.append(q_75)

  0%|          | 0/32 [00:00<?, ?it/s]

Using 3 nearest neighbors


  3%|▎         | 1/32 [00:14<07:30, 14.52s/it]

Using 3 nearest neighbors


  6%|▋         | 2/32 [00:23<05:40, 11.37s/it]

...

 94%|█████████▍| 30/32 [04:37<00:17,  8.96s/it]

Using 3 nearest neighbors


 97%|█████████▋| 31/32 [04:46<00:08,  8.97s/it]

Using 3 nearest neighbors


100%|██████████| 32/32 [04:55<00:00,  9.22s/it]


In [6]:
plot_rank_train(res_q_25, res_med, res_q_75, savepath=f"figures/{model_name}/{mode}/{size}/")